this is based on https://github.com/dimun/pate_torch/blob/master/PATE.ipynb

# Private Aggregation of Teacher Ensembles (PATE)

In [9]:
!pip freeze | grep torch

torch==1.6.0+cu101
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.7.0+cu101


## Import libraries

In [10]:
import torch

import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import Subset

## Load the [Data](http://pytorch.org/docs/stable/torchvision/datasets.html)

Downloading may take a few moments, and you should see your progress as the data is loading. You may also choose to change the `batch_size` if you want to load more data at a time.

In [11]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 32

# convert data to torch.FloatTensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))]
)

# choose the training and test datasets
train_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=transform
)
test_data = datasets.MNIST(
    root='data', train=False,
    download=True, transform=transform
)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Function for returning dataloaders for a specified number of teachers.

In [12]:
# number of teachers to essemble
num_teachers = 100

def get_data_loaders(train_data, num_teachers=10):
    """Simple partitioning algorithm that returns the right portion of the data
    needed by a given teacher out of a certain number of teachers.

    Each teacher model will get a disjoint subset of the training data.
    """
    teacher_loaders = []
    data_size = len(train_data) // num_teachers

    for i in range(num_teachers):
        indices = list(range(i * data_size, (i+1) * data_size))
        subset_data = Subset(train_data, indices)
        loader = torch.utils.data.DataLoader(
            subset_data,
            batch_size=batch_size,
            num_workers=num_workers
        )
        teacher_loaders.append(loader)

    return teacher_loaders

teacher_loaders = get_data_loaders(train_data, num_teachers)

Define a train student set of 9000 examples and 1000 test examples. Use 9000 samples of the dataset's test subset as unlabeled training points - they will be labeled using the teacher predictions.

In [13]:
student_train_data = Subset(test_data, list(range(9000)))
student_test_data = Subset(test_data, list(range(9000, 10000)))

student_train_loader = torch.utils.data.DataLoader(
    student_train_data, batch_size=batch_size, 
    num_workers=num_workers
)
student_test_loader = torch.utils.data.DataLoader(
    student_test_data, batch_size=batch_size, 
    num_workers=num_workers
)

## Defining models

We are going to define a single model for all the teachers.

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train(model, trainloader, criterion, optimizer, epochs=10, print_every=120):
    model.to(device)
    steps = 0
    running_loss = 0
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            steps += 1
            
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()


In [16]:
def predict(model, dataloader):
    outputs = torch.zeros(0, dtype=torch.long).to(device)
    model.to(device)
    model.eval()
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
    
    return outputs

## Training all the teacher models

Here we define and train the teachers

In [17]:
from tqdm.notebook import trange

# Instantiate and train the models for each teacher
def train_models(num_teachers):
    models = []
    for t in trange(num_teachers):
        model = Net()
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.003)
        train(model, teacher_loaders[t], criterion, optimizer)
        models.append(model)
    return models

models = train_models(num_teachers) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Aggregated teacher

This function predict the labels from all the dataset in each of the teachers, then return all the predictions and the maximum votation after adding laplacian noise

In [18]:
import numpy as np

In [52]:
# define standard deviation for noise
standard_deviation = 5.0

# Aggregated teacher

This function makes the predictions in all the teachers, count the votes and add noise, then returns the votation and the argmax results.

In [27]:
def aggregated_teacher(models, data_loader, standard_deviation=1.0):
    preds = torch.torch.zeros((len(models), 9000), dtype=torch.long)
    print('Running teacher predictions...')
    for i, model in enumerate(models):
        results = predict(model, data_loader)
        preds[i] = results
    
    print('Calculating aggregates...')
    labels = np.zeros(preds.shape[1]).astype(int)
    for i, image_preds in enumerate(np.transpose(preds)):
        label_counts = np.bincount(image_preds, minlength=10).astype(float)
        label_counts += np.random.normal(0, standard_deviation, len(label_counts))
        labels[i] = np.argmax(label_counts)
    
    return preds.numpy(), np.array(labels)

In [53]:
teacher_models = models
preds, student_labels = aggregated_teacher(teacher_models, student_train_loader, standard_deviation)

Running teacher predictions...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Calculating aggregates...


# Training the student

Now we will train the student with the aggregated teacher labels

In [54]:
def student_loader(student_train_loader, labels):
    for i, (data, _) in enumerate(iter(student_train_loader)):
        yield data, torch.from_numpy(labels[i*len(data):(i+1)*len(data)])

In [55]:
student_model = Net()
criterion = nn.NLLLoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.001)
epochs = 10
student_model.to(device)
steps = 0
running_loss = 0
for e in range(epochs):
    student_model.train()
    train_loader = student_loader(student_train_loader, student_labels)
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        steps += 1

        optimizer.zero_grad()
        output = student_model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if steps % 50 == 0:
            test_loss = 0
            accuracy = 0
            student_model.eval()
            with torch.no_grad():
                for images, labels in student_test_loader:
                    images, labels = images.to(device), labels.to(device)
                    log_ps = student_model(images)
                    test_loss += criterion(log_ps, labels).item()
                    
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            student_model.train()
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(student_train_loader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
            running_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/10..  Training Loss: 0.367..  Test Loss: 1.426..  Test Accuracy: 0.604
Epoch: 1/10..  Training Loss: 0.214..  Test Loss: 0.652..  Test Accuracy: 0.805
Epoch: 1/10..  Training Loss: 0.145..  Test Loss: 0.487..  Test Accuracy: 0.850
Epoch: 1/10..  Training Loss: 0.101..  Test Loss: 0.396..  Test Accuracy: 0.882
Epoch: 1/10..  Training Loss: 0.086..  Test Loss: 0.330..  Test Accuracy: 0.905
Epoch: 2/10..  Training Loss: 0.120..  Test Loss: 0.368..  Test Accuracy: 0.903
Epoch: 2/10..  Training Loss: 0.092..  Test Loss: 0.275..  Test Accuracy: 0.915
Epoch: 2/10..  Training Loss: 0.078..  Test Loss: 0.282..  Test Accuracy: 0.910
Epoch: 2/10..  Training Loss: 0.071..  Test Loss: 0.255..  Test Accuracy: 0.926
Epoch: 2/10..  Training Loss: 0.055..  Test Loss: 0.234..  Test Accuracy: 0.930
Epoch: 2/10..  Training Loss: 0.051..  Test Loss: 0.255..  Test Accuracy: 0.931
Epoch: 3/10..  Training Loss: 0.093..  Test Loss: 0.247..  Test Accuracy: 0.925
Epoch: 3/10..  Training Loss: 0.063..  T

# Privacy Analysis

In [ ]:
In Papernot and others (2018), they detail how data-dependent differential privacy bounds can be computed to estimate the cost of training the student. 

They provide a script to do this analysis based on the vote counts and the used standard deviation of the noise.

In [ ]:
!git clone https://github.com/tensorflow/privacy

In [39]:
%cd privacy/research/pate_2018/ICLR2018

In [38]:
!ls

data				    plots_for_slides.py
download.py			    rdp_bucketized.py
generate_figures.sh		    rdp_cumulative.py
generate_table_data_independent.sh  README.md
generate_table.sh		    smooth_sensitivity_table.py
plot_ls_q.py			    utility_queries_answered.py
plot_partition.py


In [42]:
preds.shape

(100, 9000)

In [56]:
# put together the counts matrix:
clean_votes = []
for image_preds in np.transpose(preds):
    label_counts = np.bincount(image_preds, minlength=10).astype(float)
    clean_votes.append(label_counts)

clean_votes = np.array(label_counts)

In [58]:
with open('clean_votes.npy', 'wb') as file_obj:
  np.save(file_obj, clean_votes)

In [45]:
#with open('labels_for_dump.npy', 'wb') as file_obj:
#  np.save(file_obj, preds)

In [57]:
standard_deviation

5.0

In [66]:
!python smooth_sensitivity_table.py  --sigma2=5.0 --counts_file=clean_votes.npy --delta=1e-5

Reading raw votes from clean_votes.npy
Shape of the votes matrix = (9000, 10)
Process all 9000 input rows. (Use --queries flag to truncate.)
queries = 1000, E[answered] = 1000.00, E[eps] = 8.862 (std = 0.83114) at order = 4.00 (contribution from delta = 3.838)
queries = 2000, E[answered] = 2000.00, E[eps] = 14.608 (std = 0.95292) at order = 3.00 (contribution from delta = 5.756)
queries = 3000, E[answered] = 3000.00, E[eps] = 18.292 (std = 0.95182) at order = 2.50 (contribution from delta = 7.675)
queries = 4000, E[answered] = 4000.00, E[eps] = 22.520 (std = 1.12738) at order = 2.50 (contribution from delta = 7.675)
queries = 5000, E[answered] = 5000.00, E[eps] = 27.469 (std = 1.30883) at order = 2.50 (contribution from delta = 7.675)
queries = 6000, E[answered] = 6000.00, E[eps] = 29.652 (std = 1.12725) at order = 2.00 (contribution from delta = 11.513)
queries = 7000, E[answered] = 7000.00, E[eps] = 30.871 (std = 1.16386) at order = 2.00 (contribution from delta = 11.513)
queries = 8

Data Independent Epsilon: 34.226

Data Dependent Epsilon: 6.998